In [ ]:
import pandas as pd
import statsmodels.api as sm
from scipy.stats import fisher_exact, zscore, gaussian_kde
from statsmodels.stats.multitest import multipletests

In [ ]:
import pandas
import os

# This query represents dataset "life quality" for domain "survey" and was generated for All of Us Controlled Tier Dataset v7
dataset_17204989_survey_sql = """
    SELECT
        answer.person_id,
        answer.question_concept_id,
        answer.question,
        answer.answer  
    FROM
        `""" + os.environ["WORKSPACE_CDR"] + """.ds_survey` answer   
    WHERE
        (
            question_concept_id IN (
                1585717, 1585723, 1585729, 1585735, 1585741, 1585747, 1585748, 1585760
            )
        )"""

dataset_17204989_survey_df = pandas.read_gbq(
    dataset_17204989_survey_sql,
    dialect="standard",
    use_bqstorage_api=("BIGQUERY_STORAGE_API_ENABLED" in os.environ),
    progress_bar_type="tqdm_notebook")

dataset_17204989_survey_df.head(5)

In [ ]:
import os

# This query represents dataset "ced2" for domain "person" and was generated for All of Us Controlled Tier Dataset v7
dataset_10934611_person_sql = """
    SELECT
        person.person_id
    FROM
        `""" + os.environ["WORKSPACE_CDR"] + """.person` person 
    LEFT JOIN
        `""" + os.environ["WORKSPACE_CDR"] + """.concept` p_gender_concept 
            ON person.gender_concept_id = p_gender_concept.concept_id 
    LEFT JOIN
        `""" + os.environ["WORKSPACE_CDR"] + """.concept` p_race_concept 
            ON person.race_concept_id = p_race_concept.concept_id 
    LEFT JOIN
        `""" + os.environ["WORKSPACE_CDR"] + """.concept` p_ethnicity_concept 
            ON person.ethnicity_concept_id = p_ethnicity_concept.concept_id 
    LEFT JOIN
        `""" + os.environ["WORKSPACE_CDR"] + """.concept` p_sex_at_birth_concept 
            ON person.sex_at_birth_concept_id = p_sex_at_birth_concept.concept_id  
    WHERE
        person.PERSON_ID IN (
            SELECT
                distinct person_id  
            FROM
                `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_person` cb_search_person  
            WHERE
                cb_search_person.person_id IN (
                    SELECT
                        criteria.person_id 
                    FROM
                        (SELECT
                            DISTINCT person_id,
                            entry_date,
                            concept_id 
                        FROM
                            `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_all_events` 
                        WHERE
                            (
                                concept_id IN (836793) 
                                AND is_standard = 0  
                                AND  value_source_concept_id IN (1384519)
                            )) criteria 
                    UNION
                    ALL SELECT
                        criteria.person_id 
                    FROM
                        (SELECT
                            DISTINCT person_id,
                            entry_date,
                            concept_id 
                        FROM
                            `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_all_events` 
                        WHERE
                            (
                                concept_id IN(
                                    SELECT
                                        DISTINCT c.concept_id 
                                    FROM
                                        `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` c 
                                    JOIN
                                        (
                                            SELECT
                                                CAST(cr.id as string) AS id       
                                            FROM
                                                `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` cr       
                                            WHERE
                                                concept_id IN (194992)       
                                                AND full_text LIKE '%_rank1]%'      
                                        ) a 
                                            ON (
                                                c.path LIKE CONCAT('%.',
                                            a.id,
                                            '.%') 
                                            OR c.path LIKE CONCAT('%.',
                                            a.id) 
                                            OR c.path LIKE CONCAT(a.id,
                                            '.%') 
                                            OR c.path = a.id) 
                                        WHERE
                                            is_standard = 1 
                                            AND is_selectable = 1
                                        ) 
                                        AND is_standard = 1 
                                )
                            ) criteria 
                        ) )"""

dataset_10934611_person_df = pandas.read_gbq(
    dataset_10934611_person_sql,
    dialect="standard",
    use_bqstorage_api=("BIGQUERY_STORAGE_API_ENABLED" in os.environ),
    progress_bar_type="tqdm_notebook")

dataset_10934611_person_df.head(5)

In [ ]:
import os

# This query represents dataset "ced2" for domain "person" and was generated for All of Us Controlled Tier Dataset v7
dataset_10934611_person_sql = """
    SELECT
        person.person_id,
        p_gender_concept.concept_name as gender,
        person.birth_datetime as date_of_birth,
        p_race_concept.concept_name as race,
        p_ethnicity_concept.concept_name as ethnicity,
        p_sex_at_birth_concept.concept_name as sex_at_birth 
    FROM
        `""" + os.environ["WORKSPACE_CDR"] + """.person` person 
    LEFT JOIN
        `""" + os.environ["WORKSPACE_CDR"] + """.concept` p_gender_concept 
            ON person.gender_concept_id = p_gender_concept.concept_id 
    LEFT JOIN
        `""" + os.environ["WORKSPACE_CDR"] + """.concept` p_race_concept 
            ON person.race_concept_id = p_race_concept.concept_id 
    LEFT JOIN
        `""" + os.environ["WORKSPACE_CDR"] + """.concept` p_ethnicity_concept 
            ON person.ethnicity_concept_id = p_ethnicity_concept.concept_id 
    LEFT JOIN
        `""" + os.environ["WORKSPACE_CDR"] + """.concept` p_sex_at_birth_concept 
            ON person.sex_at_birth_concept_id = p_sex_at_birth_concept.concept_id  
"""

person_df = pandas.read_gbq(
    dataset_10934611_person_sql,
    dialect="standard",
    use_bqstorage_api=("BIGQUERY_STORAGE_API_ENABLED" in os.environ),
    progress_bar_type="tqdm_notebook")

person_df.head(5)

In [ ]:
person_df['CeD']='healthy'
person_df.loc[person_df['person_id'].isin(dataset_10934611_person_df['person_id']),'CeD']='celiac'

In [ ]:
#calculate age
#extract enrollment date
dataset_ages_sql = """SELECT DISTINCT

person_id,

observation_datetime AS primary_consent_date

FROM `fc-aou-cdr-prod-ct.C2022Q4R11.concept`

JOIN `fc-aou-cdr-prod-ct.C2022Q4R11.concept_ancestor` on concept_id = ancestor_concept_id

JOIN `fc-aou-cdr-prod-ct.C2022Q4R11.observation` on descendant_concept_id = observation_concept_id

WHERE concept_name = 'Consent PII' AND concept_class_id = 'Module'"""

ages_df = pandas.read_gbq(
    dataset_ages_sql,
    dialect="standard",
    use_bqstorage_api=("BIGQUERY_STORAGE_API_ENABLED" in os.environ),
    progress_bar_type="tqdm_notebook")
ages_df.head(5)

In [ ]:
import os
bucket = os.getenv('WORKSPACE_BUCKET')

In [ ]:
ages_df.drop_duplicates('person_id',inplace=True)
demo_df = pandas.merge(person_df, ages_df, on='person_id', how='left')

In [ ]:
from datetime import date
def age(birthdate,enrolldate):
    age = enrolldate.year - birthdate.year - ((enrolldate.month, enrolldate.day) < (birthdate.month, birthdate.day))
    return age
from tqdm import tqdm

In [ ]:
demo_df['age'] = [age(i,j) for i,j in zip(tqdm(demo_df['date_of_birth']),tqdm(demo_df['primary_consent_date']))]

In [ ]:
demo_df=demo_df[['person_id','race','ethnicity','sex_at_birth','age','CeD']]

In [ ]:
surv_cov = pd.merge(dataset_17204989_survey_df,demo_df,on='person_id')

In [ ]:
survey_df = surv_cov.pivot(columns='question',values='answer',index=['person_id','CeD','race','ethnicity','sex_at_birth','age']).reset_index()

In [ ]:
survey_df.value_counts('CeD')

In [ ]:
survey_df.to_csv('life_quality.csv',index=False)

In [ ]:
survey_df = pd.read_csv('life_quality.csv')

In [ ]:
#matched = pd.read_csv('celiac_matched_data_non_genetic.csv')
#survey_df = survey_df[survey_df['person_id'].isin(matched_v2['person_id'])]
#survey_df

In [ ]:
#survey_df.to_csv('survey_matched_df.csv',index=False)

In [ ]:
survey_df.loc[~survey_df['sex_at_birth'].isin(['Male', 'Female','Intersex']),'sex_at_birth']='other'
survey_df.loc[~survey_df['race'].isin(['Native Hawaiian or Other Pacific Islander', 'White','Black or African American', 'Asian','Middle Eastern or North African']),'race']='other'
survey_df.loc[~survey_df['ethnicity'].isin(['Not Hispanic or Latino', 'Hispanic or Latino']),'ethnicity']='other'


In [ ]:
#first do the pain(numeric) while excluding
pain = survey_df[~survey_df['Overall Health: Average Pain 7 Days'].isin(['PMI: Skip','No matching concept'])]

pain['Overall Health: Average Pain 7 Days']=pain['Overall Health: Average Pain 7 Days'].astype(int)
pain.value_counts('CeD')

In [ ]:
survey_df.replace(['PMI: Skip','No matching concept'],[None,None],inplace=True)
survey_df.replace('General Mental Health: Excllent','General Mental Health: Excellent',inplace=True)

In [ ]:
import numpy as np
survey_df['Overall Health: Average Pain 7 Days'].fillna(np.nan,inplace=True)
survey_df['Overall Health: Average Pain 7 Days']=survey_df['Overall Health: Average Pain 7 Days'].astype(float)

In [ ]:
survey_df.columns

In [ ]:
regression_df = survey_df.copy()

In [ ]:
survey_df['Overall Health: Social Satisfaction'].unique()

In [ ]:
# Define the ordinal order
ordinal_order = ['Everyday Activities: Not At All','Everyday Activities: A Little', 'Everyday Activities: Moderately',
       'Everyday Activities: Mostly', 'Everyday Activities: Completely' ]

# Convert the categorical variable to 'category' type with the specified order
regression_df['Overall Health: Everyday Activities'] = pd.Categorical(regression_df['Overall Health: Everyday Activities'], categories=ordinal_order, ordered=True)

# Convert the categorical variable to numerical codes
regression_df['Overall Health: Everyday Activities'] = regression_df['Overall Health: Everyday Activities'].cat.codes

regression_df['Overall Health: Everyday Activities'].unique()

In [ ]:
# Define the ordinal order
ordinal_order = ['Average Fatigue 7 Days: Very Severe',   
       'Average Fatigue 7 Days: Severe',
       'Average Fatigue 7 Days: Moderate',
        'Average Fatigue 7 Days: Mild',
        'Average Fatigue 7 Days: None']

# Convert the categorical variable to 'category' type with the specified order
regression_df['Overall Health: Average Fatigue 7 Days'] = pd.Categorical(regression_df['Overall Health: Average Fatigue 7 Days'], categories=ordinal_order, ordered=True)

# Convert the categorical variable to numerical codes
regression_df['Overall Health: Average Fatigue 7 Days'] = regression_df['Overall Health: Average Fatigue 7 Days'].cat.codes
regression_df['Overall Health: Average Fatigue 7 Days'].unique()

In [ ]:
# Define the ordinal order
ordinal_order = ['Emotional Problem 7 Days: Always',
       'Emotional Problem 7 Days: Often',
       'Emotional Problem 7 Days: Sometimes',
       'Emotional Problem 7 Days: Rarely','Emotional Problem 7 Days: Never'
       ]

# Convert the categorical variable to 'category' type with the specified order
regression_df['Overall Health: Emotional Problem 7 Days'] = pd.Categorical(regression_df['Overall Health: Emotional Problem 7 Days'], categories=ordinal_order, ordered=True)

# Convert the categorical variable to numerical codes
regression_df['Overall Health: Emotional Problem 7 Days'] = regression_df['Overall Health: Emotional Problem 7 Days'].cat.codes
regression_df['Overall Health: Emotional Problem 7 Days'].unique()

In [ ]:
# Define the ordinal order
ordinal_order = ['Everyday Activities: Not At All','Everyday Activities: A Little', 'Everyday Activities: Moderately',
       'Everyday Activities: Mostly', 'Everyday Activities: Completely']

# Convert the categorical variable to 'category' type with the specified order
regression_df['cat'] = pd.Categorical(survey_df['Overall Health: Everyday Activities'], categories=ordinal_order, ordered=True)
# Convert the categorical variable to numerical codes
regression_df['Overall Health: Everyday Activities'] = regression_df['cat'].cat.codes
regression_df['Overall Health: Everyday Activities'].unique()

In [ ]:
# Define the ordinal order
ordinal_order = ['General Mental Health: Poor','General Mental Health: Fair', 'General Mental Health: Good','General Mental Health: Very Good',
        'General Mental Health: Excellent'       ]

# Convert the categorical variable to 'category' type with the specified order
regression_df['cat'] = pd.Categorical(survey_df['Overall Health: General Mental Health'], categories=ordinal_order, ordered=True)
# Convert the categorical variable to numerical codes
regression_df['Overall Health: General Mental Health'] = regression_df['cat'].cat.codes
regression_df['Overall Health: General Mental Health'].unique()

In [ ]:
# Define the ordinal order
ordinal_order = ['General Physical Health: Poor','General Physical Health: Fair',
       'General Physical Health: Good', 'General Physical Health: Very Good',
       'General Physical Health: Excellent']

# Convert the categorical variable to 'category' type with the specified order
regression_df['cat'] = pd.Categorical(survey_df['Overall Health: General Physical Health'], categories=ordinal_order, ordered=True)
# Convert the categorical variable to numerical codes
regression_df['Overall Health: General Physical Health'] = regression_df['cat'].cat.codes
regression_df['Overall Health: General Physical Health'].unique()

In [ ]:
# Define the ordinal order
ordinal_order = ['General Quality: Poor', 'General Quality: Fair', 'General Quality: Good', 'General Quality: Very Good',
       'General Quality: Excellent']

# Convert the categorical variable to 'category' type with the specified order
regression_df['cat'] = pd.Categorical(survey_df['Overall Health: General Quality'], categories=ordinal_order, ordered=True)
# Convert the categorical variable to numerical codes
regression_df['Overall Health: General Quality'] = regression_df['cat'].cat.codes
regression_df['Overall Health: General Quality'].unique()

In [ ]:
# Define the ordinal order
ordinal_order = ['Social Satisfaction: Poor', 'Social Satisfaction: Fair', 
       'Social Satisfaction: Good', 'Social Satisfaction: Very Good', 'Social Satisfaction: Excellent']

# Convert the categorical variable to 'category' type with the specified order
regression_df['cat'] = pd.Categorical(survey_df['Overall Health: Social Satisfaction'], categories=ordinal_order, ordered=True)
# Convert the categorical variable to numerical codes
regression_df['Overall Health: Social Satisfaction'] = regression_df['cat'].cat.codes
regression_df['Overall Health: Social Satisfaction'].unique()

In [ ]:
regression_df['CeD'].replace(['healthy','celiac'],[0,1],inplace=True)
cov = pd.get_dummies(regression_df[['race','ethnicity','sex_at_birth']])

In [ ]:
cov=cov.astype(int)

In [ ]:
cov=cov[['race_Asian', 'race_Black or African American',
       'race_Middle Eastern or North African',
       'race_Native Hawaiian or Other Pacific Islander', 'race_White','ethnicity_Hispanic or Latino','sex_at_birth_Female', 'sex_at_birth_Intersex', 'sex_at_birth_Male',]]

In [ ]:
regress = pd.concat([regression_df,cov],axis=1).drop(['cat','race','ethnicity','sex_at_birth'],axis=1)

In [ ]:
regress.replace(-1, np.nan, inplace=True)

In [ ]:
regress

In [ ]:
# Preparing the dataset for regression
X = regress.drop(['person_id','CeD'],axis=1)
X = sm.add_constant(X)  # Adding a constant for the intercept
y = regress['CeD'] # Adjusting Celiac variable to binary

In [ ]:
# Performing the logistic regression
#need to filter nans in pain first
model = sm.Logit(y, X, missing = 'drop').fit()

In [ ]:
model.pvalues.to_csv('life_quality_pvals_matched.csv')

In [ ]:
regress.loc[~regress.isna().any(axis=1),'CeD'].value_counts()

In [ ]:
# Exporting the summary to a text file
summary_str = model.summary().as_text()
summary_output_path = 'life_quality_logistic_regression_summary_matched.txt'
with open(summary_output_path, 'w') as file:
    file.write(summary_str)

In [ ]:
model.summary() #no person has duplicate answers for any question

In [ ]:
regress['Overall Health: Average Pain 7 Days'].replace([0,1,2,3,4,5,6,7,8,9,10],[1,2,2,2,3,3,3,4,4,4,5],inplace=True)

In [ ]:
regress[['Overall Health: Emotional Problem 7 Days',
       'Overall Health: Everyday Activities','Overall Health: Average Fatigue 7 Days',
       'Overall Health: General Mental Health',
       'Overall Health: General Physical Health',
       'Overall Health: General Quality',
       'Overall Health: Social Satisfaction']]+=1

In [ ]:
regress#['Overall Health: Average Fatigue 7 Days'].unique()

In [ ]:
regress['PH_raw']=(6-regress['Overall Health: Average Pain 7 Days'])+regress['Overall Health: Everyday Activities']+regress['Overall Health: General Physical Health']+regress['Overall Health: Average Fatigue 7 Days']

In [ ]:
regress['MH_raw']=regress['Overall Health: General Mental Health']+regress['Overall Health: General Quality']+regress['Overall Health: Social Satisfaction']+regress['Overall Health: Emotional Problem 7 Days']

In [ ]:
regress['Global_raw']=regress['PH_raw']+regress['MH_raw']

In [ ]:
#T-score with mean 50 and std 10
def tscore(arr):
    val = zscore(arr)*10+50
    return val

In [ ]:
regress.loc[~pd.isna(regress['PH_raw']),'PROMIS-PH']=tscore(regress['PH_raw'].dropna())

In [ ]:
regress.loc[~pd.isna(regress['MH_raw']),'PROMIS-MH']=tscore(regress['MH_raw'].dropna())

In [ ]:
regress.loc[~pd.isna(regress['Global_raw']),'PROMIS-total']=tscore(regress['Global_raw'].dropna())

In [ ]:
#combined score regression
X2 = regress[['age','race_Asian',
       'race_Black or African American',
       'race_Middle Eastern or North African',
       'race_Native Hawaiian or Other Pacific Islander', 'race_White',
       'ethnicity_Hispanic or Latino', 'sex_at_birth_Female',
       'sex_at_birth_Intersex', 'sex_at_birth_Male','CeD']]
X2 = sm.add_constant(X2)  # Adding a constant for the intercept
y = regress['PROMIS-total']

In [ ]:
# Performing the logistic regression
model_agg = sm.OLS(y, X2, missing = 'drop').fit()

In [ ]:
# Exporting the summary to a text file
summary_str = model_agg.summary().as_text()
summary_output_path = 'PROMIS_score_linear_regression_summary_matched.txt'
with open(summary_output_path, 'w') as file:
    file.write(summary_str)
!gsutil cp PROMIS_score_linear_regression_summary_matched.txt $WORKSPACE_BUCKET/data/life_quality/

In [ ]:
model_agg.pvalues.to_csv('PROMIS_score_linear_pvals_matched.csv')
!gsutil cp PROMIS_score_linear_pvals_matched.csv $WORKSPACE_BUCKET/data/life_quality/

In [ ]:
regress=pd.read_csv(f'{bucket}/data/life_quality/PROMIS_scores.csv')
matched_v2 = pd.read_csv(f'{bucket}/data/gwas_v2/phenotypes/celiac_matched_data_v2.csv')

In [ ]:
first_match=regress[regress['person_id'].isin(matched_v2.person_id)]

In [ ]:
first_match.loc[~pd.isna(first_match['PH_raw']),'PROMIS-PH']=tscore(first_match['PH_raw'].dropna())
first_match.loc[~pd.isna(first_match['MH_raw']),'PROMIS-MH']=tscore(first_match['MH_raw'].dropna())

first_match.loc[~pd.isna(first_match['Global_raw']),'PROMIS-total']=tscore(first_match['Global_raw'].dropna())

In [ ]:
first_match.to_csv('PROMIS_scores_gen_matched_v2.csv',index=False)
!gsutil cp PROMIS_scores_gen_matched_v2.csv $WORKSPACE_BUCKET/data/life_quality/

In [ ]:
regress.to_csv('PROMIS_scores.csv',index=False)
!gsutil cp PROMIS_scores.csv $WORKSPACE_BUCKET/data/life_quality/

In [ ]:

regress=pd.read_csv(f'{bucket}/data/life_quality/PROMIS_scores.csv')

In [ ]:
regress[~pd.isna(regress['PROMIS-total'])].value_counts('CeD')

In [ ]:
regress[~regress[['Overall Health: Average Fatigue 7 Days',
       'Overall Health: Average Pain 7 Days',
       'Overall Health: Emotional Problem 7 Days',
       'Overall Health: Everyday Activities',
       'Overall Health: General Mental Health',
       'Overall Health: General Physical Health',
       'Overall Health: General Quality',
       'Overall Health: Social Satisfaction']].isna().all(axis=1)]